<a href="https://colab.research.google.com/github/Aman-Singh-Akhand/QML-HEP-GSoC-2025/blob/main/Task_VI_Quantum_representation_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit torch torchvision qiskit-machine-learning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y qiskit qiskit-aer qiskit-terra qiskit-ibm-provider


Found existing installation: qiskit 1.4.2
Uninstalling qiskit-1.4.2:
  Successfully uninstalled qiskit-1.4.2
Found existing installation: qiskit-aer 0.17.0
Uninstalling qiskit-aer-0.17.0:
  Successfully uninstalled qiskit-aer-0.17.0


In [ ]:
!pip install qiskit
!pip install qiskit-aer


In [ ]:
!pip install qiskit qiskit-aer --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.4 MB/s eta 0:00:00


In [ ]:
import qiskit

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from sklearn.model_selection import train_test_split

# Loading MNIST Dataset
transform = transforms.Compose([transforms.ToTensor()])
mnist_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Converting to NumPy arrays
X = mnist_dataset.data.numpy()
y = mnist_dataset.targets.numpy()
X_test = test_dataset.data.numpy()
y_test = test_dataset.targets.numpy()

# Downsample Images to reduce the computational cost
def downsample_image(image, new_size=(2, 4)):  # Reduced to 28x28 to 2x4 = 8 pixels
    image = torch.tensor(image, dtype=torch.float32).view(1, 1, 28, 28)
    downsampled = F.avg_pool2d(image, kernel_size=(14, 7))
    return downsampled.flatten()

# Downsampling
X_reduced = np.array([downsample_image(img) for img in X])
X_test_reduced = np.array([downsample_image(img) for img in X_test])

# Normalize images for quantum encoding
def preprocess_for_quantum(image):
    return (image - image.min()) / (image.max() - image.min()) * np.pi  # Scale to [0, π]

X_quantum = np.array([preprocess_for_quantum(img) for img in X_reduced])
X_test_quantum = np.array([preprocess_for_quantum(img) for img in X_test_reduced])

In [ ]:
# train & validation sets
X_train, X_val, y_train, y_val = train_test_split(X_quantum, y, test_size=0.2, random_state=42)

# Quantum Circuit
n_qubits = 8  # Number of qubits here matches downsampled image pixels

def quantum_embedding(qc, image, params):
    """Embed an image as a quantum state with trainable parameters."""
    for i in range(len(image)):
        qc.rx(params[i], i)  # Parameterized rotation has been used here

def swap_test_circuit(image1, image2, params1, params2, n_qubits):
    """Perform a SWAP test to compute quantum state fidelity."""
    qc = QuantumCircuit(n_qubits + 1, 1)  # Extra qubit for SWAP test (ancilla)

    # Embed the images into quantum states
    quantum_embedding(qc, image1, params1)
    quantum_embedding(qc, image2, params2)

    # Hadamard gate on the ancilla qubit (used as the control)
    qc.h(n_qubits)

    # Applying controlled SWAP gates between different qubits
    for i in range(n_qubits):
        qc.cswap(n_qubits, i, (i + 1) % n_qubits)  # Correctly handle target qubits

    # Apply Hadamard again on the ancilla qubit
    qc.h(n_qubits)

    # Measure the ancilla qubit into the classical register
    qc.measure(n_qubits, 0)

    return qc

# Compute Fidelity using Quantum Simulation
def get_fidelity(image1, image2, params1, params2, n_qubits):
    """Run SWAP test and return fidelity score."""
    image1 = np.array(image1)
    image2 = np.array(image2)
    params1 = np.array(params1)
    params2 = np.array(params2)

    qc = swap_test_circuit(image1, image2, params1, params2, n_qubits)  # Pass n_qubits
    simulator = AerSimulator()
    transpiled_circuit = transpile(qc, simulator)
    result = simulator.run(transpiled_circuit, shots=1024).result()

    counts = result.get_counts()
    p0 = counts.get('0', 0) / 1024  # Probability of measuring |0⟩
    return p0  # Fidelity estimate

# Contrastive Loss Function
def contrastive_loss(fidelity, label_same):
    """Loss function to maximize fidelity for same-class images and minimize otherwise."""
    loss_value = (1 - fidelity) ** 2 if label_same else fidelity ** 2
    return torch.tensor(loss_value, requires_grad=True)  # This ensures that the loss is a tensor






In [ ]:
# Training the Quantum Circuit
params1 = torch.randn(n_qubits, requires_grad=True)  # Trainable params
params2 = torch.randn(n_qubits, requires_grad=True)  # Trainable params
optimizer = optim.Adam([params1, params2], lr=0.01)

num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(len(X_train) - 1):
        image1, image2 = X_train[i], X_train[i + 1]
        label1, label2 = y_train[i], y_train[i + 1]
        label_same = 1 if label1 == label2 else 0

        # Convert params1 and params2 to numpy for quantum processing
        fidelity = get_fidelity(image1, image2, params1.detach().numpy(), params2.detach().numpy(), n_qubits)
        loss = contrastive_loss(fidelity, label_same)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(X_train):.4f}")

Epoch 1/5, Loss: 0.3880
Epoch 2/5, Loss: 0.3880
Epoch 3/5, Loss: 0.3881
Epoch 4/5, Loss: 0.3880
Epoch 5/5, Loss: 0.3880


In [ ]:
# Evaluate model on Test Data
correct = 0
total = len(X_test_quantum) - 1

for i in range(total):
    image1, image2 = X_test_quantum[i], X_test_quantum[i + 1]
    label1, label2 = y_test[i], y_test[i + 1]
    label_same = 1 if label1 == label2 else 0

    # Convert params1 and params2 to numpy for quantum processing
    fidelity = get_fidelity(image1, image2, params1.detach().numpy(), params2.detach().numpy(), n_qubits)
    prediction = 1 if fidelity > 0.5 else 0  # Assume threshold 0.5

    if prediction == label_same:
        correct += 1

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.0845


In [ ]:
import qiskit
from qiskit_aer import Aer

print("Qiskit version:", qiskit.__version__)
print("Aer backend available:", Aer.backends())


Qiskit version: 1.4.2
Aer backend available: [AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator')]
